<a href="https://colab.research.google.com/github/steinhaug/stable-diffusion/blob/main/stable/SD_Inference_with_Lora.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from google.colab import userdata
HF_TOKEN = userdata.get('HF_TOKEN')

!huggingface-cli login --token {HF_TOKEN}
from huggingface_hub import snapshot_download

In [ ]:
#@ markdown Download image sets
import os
SAVE_PATH = '/content/drive/MyDrive/models/SD'
REPO_ID = 'stablediffusionapi/deliberate-v2'
os.makedirs(f"{SAVE_PATH}/{REPO_ID}", exist_ok=True)
path = snapshot_download(repo_id=REPO_ID, repo_type="model", revision="main", local_dir=f"{SAVE_PATH}/{REPO_ID}", local_dir_use_symlinks=False)

### Install deps

In [ ]:
!pip install accelerate
!pip install peft
!pip install -U git+https://github.com/huggingface/diffusers.git@main

### Inference

__Load LoRA using Diffusers LoraLoaderMixin__  
Assume we have the LoRA file in safetensor file format, to load LoRA using Diffusers is as easy as this.

In [ ]:
to_mode = 'cuda:0'
SAVE_PATH = '/content/drive/MyDrive/models/SD'

In [ ]:
import torch
from diffusers import StableDiffusionPipeline

text2img_pipe = StableDiffusionPipeline.from_pretrained(
    f"{SAVE_PATH}/stablediffusionapi/deliberate-v2"
    , torch_dtype = torch.float16
    , safety_checker = None
).to(to_mode)

In [ ]:
lora_path = "/content/drive/MyDrive/Loras/skinny_hot_girls/output_64_32/skinny_hot_girls-10.safetensors"
text2img_pipe.load_lora_weights(lora_path)

Just one line code: text2img_pipe.load_lora_weights(lora_path). Test with one of the famous LoRA — LowRA , this LoRA can turn the image to dark mode.

Let’s test it with and without LoRA loaded.

In [ ]:
from diffusers import EulerDiscreteScheduler

prompt = """
Naked Ambition. A young woman with a rose in her hand and a pink flower in her mouth. she is wearing white stockings and is sitting on a wooden table. Nswf, totally naked. Shaved pussy. Small tits, small breasts. Normal figure.
"""
neg_prompt = """
NSFW,deformed, distorted, disfigured, poorly drawn, bad anatomy, wrong anatomy, extra limb, missing limb, floating limbs, mutated hands and fingers, disconnected limbs, mutation, mutated, ugly, disgusting, blurry, amputation
"""

text2img_pipe.scheduler = EulerDiscreteScheduler.from_config(text2img_pipe.scheduler.config)

image = text2img_pipe(
    prompt = prompt
    , negative_prompt = neg_prompt
    , generator = torch.Generator(to_mode).manual_seed(3135098381)
    , num_inference_steps = 28
    , guidance_scale = 8
    , width = 512
    , height = 768
).images[0]
display(image)

__Load LoRA using Diffusers with a weight__  
Even though we can now load LoRA with just one line code, we still don’t see any support for the LoRA weight parameter. For example, we want less dark from the result and say add LoRA weight as 0.5. In A1111 Stable Diffusion Webui, we can give the LoRA weight as: <lora:LowRA:0.5>

How can we add this 0.5 to the Diffusers package? seems there is no way, but we can hack into the process and add our value in the middle of the LoRA loading like this:

In [ ]:
text2img_pipe.unload_lora_weights()
lora_path = "<path/to/lora.safetensors>"

lora_w = 0.5
text2img_pipe._lora_scale = lora_w

state_dict, network_alphas = text2img_pipe.lora_state_dict(
    lora_path
)

for key in network_alphas:
    network_alphas[key] = network_alphas[key] * lora_w

#network_alpha = network_alpha * lora_w
text2img_pipe.load_lora_into_unet(
    state_dict = state_dict
    , network_alphas = network_alphas
    , unet = text2img_pipe.unet
)

text2img_pipe.load_lora_into_text_encoder(
    state_dict = state_dict
    , network_alphas = network_alphas
    , text_encoder = text2img_pipe.text_encoder
)

To reset the LoRA, simply call the unload_lora_weights function :

In [ ]:
text2img_pipe.unload_lora_weights()